# Stock Stalker 使用示例

本Notebook演示Stock Stalker的核心功能

In [ ]:
import sys
sys.path.insert(0, '.')

from datetime import datetime
import json

## 1. 扫描股票信号

In [ ]:
from core.watchlist_scanner import WatchlistScanner

scanner = WatchlistScanner()

# 添加股票到观察列表
scanner.add_ticker("AAPL", "Apple Inc.")
scanner.add_ticker("TSLA", "Tesla Inc.")
scanner.add_ticker("NVDA", "NVIDIA Corp")

print(f"观察列表: {list(scanner.items.keys())}")

In [ ]:
# 扫描单个股票
result = scanner.scan_ticker("AAPL", save_to_db=True)

print(f"股票: {result['ticker']}")
print(f"状态: {result['state']}")
print(f"确定性评分: {result['conviction']['score']}/100")

if 'trade_plan' in result:
    plan = result['trade_plan']
    print(f"\n交易计划:")
    print(f"  方向: {plan.get('direction')}")
    print(f"  入场: {plan.get('entry_price')}")
    print(f"  止损: {plan.get('stop_loss')}")
    print(f"  止盈: {plan.get('take_profit')}")

## 2. 技术分析

In [ ]:
from analysis.technical_indicators import TechnicalAnalyzer, TechnicalIndicators
import random

# 生成示例价格数据
prices = []
volumes = []
highs = []
lows = []

base_price = 100
for i in range(100):
    change = random.gauss(0, 2)
    base_price += change
    base_price = max(base_price, 50)
    
    prices.append(base_price)
    volumes.append(random.randint(1000000, 5000000))
    highs.append(base_price + random.uniform(0, 3))
    lows.append(base_price - random.uniform(0, 3))

print(f"生成 {len(prices)} 天价格数据")
print(f"价格范围: ${min(prices):.2f} - ${max(prices):.2f}")

In [ ]:
# 计算技术指标
indicators = TechnicalAnalyzer.calculate_all(
    ticker="AAPL",
    prices=prices,
    volumes=volumes,
    highs=highs,
    lows=lows
)

print("\n📈 趋势指标:")
print(f"  SMA 20: ${indicators.sma_20:.2f}")
print(f"  SMA 50: ${indicators.sma_50:.2f}")

print("\n💹 动量指标:")
print(f"  RSI 14: {indicators.rsi_14:.2f}")
print(f"  MACD: {indicators.macd:.4f}")

print("\n📊 波动率指标:")
print(f"  布林带上轨: ${indicators.bb_upper:.2f}")
print(f"  布林带下轨: ${indicators.bb_lower:.2f}")

print(f"\n🎯 交易信号: {indicators.get_signal()}")

## 3. 策略回测

In [ ]:
from analysis.backtest import BacktestEngine, simple_ma_strategy, rsi_strategy
from datetime import datetime, timedelta

# 准备历史数据
price_data = []
price = 100.0

for i in range(100):
    change = random.gauss(0.001, 0.02)
    price = price * (1 + change)
    
    price_data.append({
        'date': datetime.now() - timedelta(days=100-i),
        'open': price * (1 + random.gauss(0, 0.005)),
        'high': price * (1 + abs(random.gauss(0, 0.01))),
        'low': price * (1 - abs(random.gauss(0, 0.01))),
        'close': price,
        'volume': random.randint(1000000, 5000000),
    })

print(f"回测数据: {len(price_data)} 天")

In [ ]:
# 使用均线策略回测
engine = BacktestEngine(initial_capital=100000.0)
result_ma = engine.run_backtest(
    ticker="AAPL",
    price_data=price_data,
    strategy=simple_ma_strategy,
)

print(result_ma.summary())

In [ ]:
# 使用RSI策略回测
engine2 = BacktestEngine(initial_capital=100000.0)
result_rsi = engine2.run_backtest(
    ticker="AAPL",
    price_data=price_data,
    strategy=rsi_strategy,
)

print(result_rsi.summary())

In [ ]:
# 策略对比
print("\n" + "="*60)
print("策略对比")
print("="*60)
print(f"{\"指标\":<25} {\"均线策略\":>15} {\"RSI策略\":>15}")
print("-"*60)
print(f"{\"总收益率\":<25} {result_ma.total_return_pct:>14.2f}% {result_rsi.total_return_pct:>14.2f}%")
print(f"{\"交易次数\":<25} {result_ma.total_trades:>15} {result_rsi.total_trades:>15}")
print(f"{\"胜率\":<25} {result_ma.win_rate:>14.1%} {result_rsi.win_rate:>14.1%}")
print(f"{\"最大回撤\":<25} {result_ma.max_drawdown_pct:>14.2f}% {result_rsi.max_drawdown_pct:>14.2f}%")
print(f"{\"夏普比率\":<25} {result_ma.sharpe_ratio:>15.2f} {result_rsi.sharpe_ratio:>15.2f}")
print("="*60)

## 4. 通知系统

In [ ]:
from core.notifications import create_notification_manager

# 创建通知管理器（仅控制台）
manager = create_notification_manager(console=True)

# 发送交易信号通知
manager.send_trade_signal(
    ticker="AAPL",
    action="BUY",
    price=150.25,
    conviction=85,
    reason="RSI oversold, MACD bullish crossover",
    priority="high",
)

In [ ]:
# 发送自定义提醒
manager.send_alert(
    title="市场提醒",
    body="标普500过去一小时下跌2%",
    priority="urgent",
)

In [ ]:
# 查看通知历史
history = manager.get_history(limit=5)
print(f"\n最近 {len(history)} 条通知:")
for msg in history:
    print(f"  - [{msg.priority}] {msg.title}")

## 5. 增强扫描器

In [ ]:
from core.enhanced_scanner import EnhancedWatchlistScanner

enhanced = EnhancedWatchlistScanner()

# 扫描股票（含技术指标）
result = enhanced.scan_with_technicals("AAPL")

print(f"股票: {result['ticker']}")
print(f"确定性评分: {result['conviction']['score']}/100")

if 'technical_indicators' in result:
    print(f"\n技术指标:")
    ti = result['technical_indicators']
    print(f"  RSI: {ti.get('rsi_14', 'N/A')}")
    print(f"  MACD: {ti.get('macd', 'N/A')}")

print(f"\n技术信号: {result.get('technical_signal', 'N/A')}")

## 6. 数据导出

In [ ]:
import json

# 导出回测结果
backtest_data = {
    'ma_strategy': result_ma.to_dict(),
    'rsi_strategy': result_rsi.to_dict(),
}

with open('backtest_comparison.json', 'w') as f:
    json.dump(backtest_data, f, indent=2, default=str)

print("✅ 回测结果已导出到 backtest_comparison.json")

## 总结

本Notebook演示了Stock Stalker的核心功能：

1. ✅ 股票扫描和信号检测
2. ✅ 技术指标分析
3. ✅ 策略回测
4. ✅ 通知系统
5. ✅ 数据导出

更多信息请参考 README.md